# Running EDHOC with the public demo server

This demo can be run on the public Jupyter instance at [try Jupyter](https://jupyter.org/try-jupyter/lab/index.html) by uploading this file.

In [1]:
import micropip, pyodide

if pyodide.__version__ == "0.26.2":  # get it from the nightl
    await micropip.install(
        "https://cdn.jsdelivr.net/pyodide/dev/full/lakers_python-0.3.3-cp312-cp312-pyodide_2024_0_wasm32.whl"
    )
await micropip.install("aiocoap[oscore,prettyprint]")

In [2]:
from aiocoap import *
from aiocoap import message, edhoc, transports

ctx = await Context.create_client_context()

In [3]:
ephemeral = edhoc.CoseKeyForEdhoc.generate()
for_demo = {
    "coap://demo.coap.amsuess.com/*": {
        "edhoc-oscore": {
            "suite": 2,
            "method": 3,
            "own_cred_style": "by-value",
            # with current lakers, we need kid and scope even if we don't access them anywhere
            "own_cred": ephemeral.as_ccs(kid="\x01", subject="2"),
            "private_key": ephemeral.secret_to_map(),
            "peer_cred": {
                14: {
                    2: "demo.coap.amsuess.com",
                    8: {
                        1: {
                            1: 2,
                            2: b"\0",
                            -1: 1,
                            -2: bytes.fromhex(
                                "b9cc746df6641d55044478b29df019ef22b4d2e96ffcf8de85434e5d0f27c33c"
                            ),
                            -3: bytes.fromhex(
                                "e14e87330d093b469b121c3d0e4d9452cb90036a6e209f21f37d35d2a05c426c"
                            ),
                        }
                    },
                }
            },
        }
    }
}
ctx.client_credentials.load_from_dict(for_demo)

In [5]:
req = Message(code=GET, uri="coap://demo.coap.amsuess.com/whoami")
# FIXME: We should have a prettier way to set a proxy
req.opt.proxy_scheme = "coap"
req.remote = message.UndecidedRemote.from_pathless_uri(
    "coaps+ws://proxy.coap.amsuess.com"
)
res = await ctx.request(req).response
res

<aiocoap.Message at 0x356bb28: no mtype, 2.05 Content (no MID, empty token) remote <OSCOREAddress in context <aiocoap.edhoc.EdhocInitiatorContext object at 0xfb8e00> to <WSRemote at 0x12b3d78, hostinfo proxy.coap.amsuess.com, local localhost>>, 1 option(s), 164 byte(s) payload>